<a href="https://colab.research.google.com/github/davidsonsantana89/dass--CESAR_SCHOOL-Topicos_Contemporaneos/blob/main/exercicios/parte_2/dass_09_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DAVIDSON ALVES SANTOS DE SANTANA

dass@cesar.school

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


## Retrieval Augmented Generation (RAG)

In [ ]:
!pip install langchain_community langchain_openai langchain_text_splitters langchain_community faiss-gpu

### Carregando Documentos - Loading

In [ ]:
# https://python.langchain.com/v0.2/docs/how_to/#document-loaders
# https://python.langchain.com/v0.2/docs/integrations/document_loaders/

import bs4
from langchain_community.document_loaders import WebBaseLoader, TextLoader

# Filtra o conteúdo da página por uma classe específica
bs4_strainer = bs4.SoupStrainer(class_=("mw-body-content"))

# Carrega o conteúdo da página
loader = WebBaseLoader(
    web_paths=("https://pt.wikipedia.org/wiki/Centro_de_Estudos_e_Sistemas_Avan%C3%A7ados_do_Recife",),
    bs_kwargs={"parse_only": bs4_strainer},
)

# Carrega o conteúdo da página
docs = loader.load()

len(docs[0].page_content)

6915

In [ ]:
print(docs[0].page_content[500:1000])

cife, também conhecido por seu acrônimo CESAR, é um centro de pesquisa e inovação sem fins lucrativos com sede na cidade do Recife, Pernambuco e filiais em Sorocaba, Curitiba e Manaus.[1] O CESAR foi fundado em 1996 por três professores do Centro de Informática da UFPE, Silvio Meira, Fábio Silva e Ismar Kaufman, como forma de aproximar a academia do mercado.[2]
Em 2019, o centro conta com mais de 600 funcionários e em 2018 seu faturamento foi da ordem de R$ 100 milhões.[1][3][4][5]
O CESAR é par


### Dividindo Documentos - Splitting/Chunking

In [ ]:
# https://python.langchain.com/v0.2/docs/how_to/#text-splitters

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

10

In [ ]:
print(all_splits[2].page_content)

Área Educacional[editar | editar código-fonte]
Além de atuar como centro de pesquisa e inovação, o CESAR criou um braço educacional, a CESAR School, oferecendo cursos de graduação, mestrados e doutorados profissionais.[4]
O centro iniciou sua atuação na área educacional em 2007 quando iniciou a oferta do mestrado profissional em Engenharia de Software, que foi avaliado pela CAPES em 2017 como um dos dois melhores mestrados profissionais na área de computação do país.[8] Em 2013 foi autorizada a abertura do segundo mestrado profissional, dessa vez com ênfase em Design de Artefatos Digitais.[9] A partir de 2016 o mestrado em Design passou a também ser oferecido na unidade de Manaus.[10]


### Indexando - Store

In [ ]:
# https://python.langchain.com/v0.2/docs/how_to/embed_text/

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("Onde está localizado o CESAR?")

len(retrieved_docs)

6

In [ ]:
print(retrieved_docs[0].page_content)

Área Educacional[editar | editar código-fonte]
Além de atuar como centro de pesquisa e inovação, o CESAR criou um braço educacional, a CESAR School, oferecendo cursos de graduação, mestrados e doutorados profissionais.[4]
O centro iniciou sua atuação na área educacional em 2007 quando iniciou a oferta do mestrado profissional em Engenharia de Software, que foi avaliado pela CAPES em 2017 como um dos dois melhores mestrados profissionais na área de computação do país.[8] Em 2013 foi autorizada a abertura do segundo mestrado profissional, dessa vez com ênfase em Design de Artefatos Digitais.[9] A partir de 2016 o mestrado em Design passou a também ser oferecido na unidade de Manaus.[10]


### Buscando e Recuperando Informações - Retrieve

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [ ]:
example_messages = prompt_template.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages.to_messages())

[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.\n\nPergunta: alguma pergunta\n\nContexto: algum contexto\n\nResposta:\n')]


### Gerando Respostas - Generate

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [ ]:
for chunk in rag_chain.stream("Onde fica o CESAR?"):
    print(chunk, end="", flush=True)

O CESAR, ou Centro de Estudos e Sistemas Avançados do Recife, fica na cidade do Recife, Pernambuco. Ele também possui filiais em Sorocaba, Curitiba e Manaus.

## Exercícios

### Exercício 1
Faça um RAG com um pequeno arquivo de texto, contendo informações que, certamente, a LLM não conheça. Você deverá construir o arquivo e enviar para o ambiente de execução. Escolha a forma de chunking apropriada para o seu documento.

O texto escolhido para esta atividade foi a Instrução Normativa de Avaliação do Estado de Pernambuco Nº 04\/2014.

In [ ]:
!mkdir data

In [ ]:
!wget https://raw.githubusercontent.com/davidsonsantana89/dass--CESAR_SCHOOL-Topicos_Contemporaneos/main/arquivos_de_texto/INSTRU%C3%87%C3%83O%20NORMATIVA%20DE%20AVALIA%C3%87%C3%83O%20N%C2%BA%2004-2014.txt -O data/instrucao_normativa.txt

--2024-08-31 20:04:21--  https://raw.githubusercontent.com/davidsonsantana89/dass--CESAR_SCHOOL-Topicos_Contemporaneos/main/arquivos_de_texto/INSTRU%C3%87%C3%83O%20NORMATIVA%20DE%20AVALIA%C3%87%C3%83O%20N%C2%BA%2004-2014.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22581 (22K) [text/plain]
Saving to: ‘data/instrucao_normativa.txt’

data/instrucao_norm 100%[===================>]  22.05K  --.-KB/s    in 0.002s  

2024-08-31 20:04:21 (13.4 MB/s) - ‘data/instrucao_normativa.txt’ saved [22581/22581]



In [ ]:
caminho_arquivo = "data/instrucao_normativa.txt"

loader = TextLoader(caminho_arquivo)
docs = loader.load()

print(docs[0].page_content[:1000])

 INSTRUÇÃO NORMATIVA DE AVALIAÇÃO Nº 04/2014
 Dispõe sobre as diretrizes e procedimentos do Sistema de Avaliação das Aprendizagens nas Escolas da Rede Estadual de Ensino, a 
partir do ano letivo de 2015.
 O SECRETÁRIO DE EDUCAÇÃO E ESPORTES, no uso da competência que lhe foi conferida pelo Decreto Estadual nº 40.599/2014, 
por intermédio da Secretaria Executiva de Desenvolvimento da Educação - SEDE, através da Gerência de Normatização do Ensino - 
GENE, com base na Lei Federal nº. 9.394/96, alterada pela Lei Federal n°12.796/13, na Lei nº 10.172/01, na Lei Federal nº 11.274/06, 
no Parecer CNE/CEB nº 6/05, na Resolução CNE/CEB nº 3/05, no Parecer CNE/CEB nº 18/05, na Resolução CEE/PE nº 03/06, na Lei 
Estadual nº 12.252/02, na Lei Estadual nº 12.286/02, na Lei Estadual nº 14.789/12, na Resolução CEE/PE nº 02/07, no Parecer CNE/
 CEB nº 20/09, na Resolução CNE/CEB nº 5/09, no Parecer CNE/CEB nº 5/11, na Resolução CNE/CEB nº 2/12, no Decreto Federal nº 
5.154/04, no Decreto Federal nº 5.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
todos_pedacos = text_splitter.split_documents(docs)

len(todos_pedacos)

27

In [ ]:
print(todos_pedacos[15].page_content)

do Normal em nível Médio, da EJA Médio e do PROEJA, quando concluir com êxito o 9º ano do Ensino Fundamental.
 § 5º O (a) estudante em regime de progressão parcial deverá obter em cada componente curricular a nota mínima 6,0 (seis) para 
aprovação.
 § 6º Ao (à) estudante em regime de progressão parcial, serão oferecidas, no mínimo, 03 (três) oportunidades de reensino e avaliação da 
aprendizagem no ano letivo subsequente.
 § 7º O (A) estudante reprovado (a) em até dois componentes curriculares no 9º ano e na IV fase da EJA do Ensino Fundamental, no 
3º ano do Ensino Médio, do Ensino Médio Integral, do Ensino Médio Semi-Integral, no 4º ano do Ensino Médio Integrado à Educação 
Profissional, no 4º ano da Educação Profissional em nível Médio, no 4º ano do Normal em nível Médio, no 3º módulo da EJA Médio e no 4º 
módulo do PROEJA, terá direito a exame especial de progressão parcial, a realizar-se no final do ano/fase/módulo letivo, conferindo-lhe,


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vectorstore = FAISS.from_documents(todos_pedacos, OpenAIEmbeddings())

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("Quando ocorre a progressão plena?")

len(retrieved_docs)

6

In [ ]:
print(retrieved_docs[1].page_content)

no Projeto Político - Pedagógico, por unidades didáticas bimestrais, considerando os conteúdos curriculares definidos pela Secretaria de 
Educação e Esportes.
 Art. 4º Os critérios avaliativos deverão ser estabelecidos a partir dos conteúdos definidos pela Secretaria de Educação e Esportes para 
cada componente curricular, com base nas Orientações Teórico-Metodológicas - OTMs.
 Art. 5º O (a) estudante, ao longo da sua escolaridade, poderá obter progressão plena ou parcial:
 I - a progressão plena dar-se-á quando o (a) estudante atingir ao término do ano letivo ou após o período de recuperação final, nota 
igual ou superior a 6,0 (seis) em todos os componentes curriculares do (a) ano/série/ciclo/fase/módulo/ ano de escolaridade e frequência 
mínima de 75% (setenta e cinco por cento) do total das horas letivas do (a) ano/série/ciclo/fase/módulo/ano de escolaridade;


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [ ]:
example_messages = prompt_template.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages.to_messages())

[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.\n\nPergunta: alguma pergunta\n\nContexto: algum contexto\n\nResposta:\n')]


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [65]:
for chunk in rag_chain.stream("Quando ocorre a progressão parcial?"):
    print(chunk, end="", flush=True)

A progressão parcial ocorre quando o estudante, após o período de recuperação final, não obtiver aprovação em até dois componentes curriculares e atende a certas condições estabelecidas pela escola. É um direito do estudante e pode ser aplicada em diversos anos e modalidades de ensino.

In [ ]:
for chunk in rag_chain.stream("Como deve ser o registro de notas?"):
    print(chunk, end="", flush=True)

O registro de notas deve ser feito em uma escala de 0,0 a 10,0, mantendo até uma casa decimal, e deve ser concluído até cinco dias úteis após o término da unidade didática ou bimestre. O arredondamento das notas, quando necessário, será sempre por acréscimo.

In [64]:
for chunk in rag_chain.stream("Como é a avaliação bimestral?"):
    print(chunk, end="", flush=True)

A avaliação bimestral consiste em duas notas: a 1ª, que resulta de atividades diversas durante o bimestre, e a 2ª, que corresponde a uma avaliação individual no final do bimestre. A média é calculada somando as duas notas e dividindo por dois, sendo necessário alcançar no mínimo 6,0 para aprovação.

In [ ]:
for chunk in rag_chain.stream("Quando ocorre a progressão plena?"):
    print(chunk, end="", flush=True)

A progressão plena ocorre quando o estudante atinge, ao término do ano letivo ou após o período de recuperação final, nota igual ou superior a 6,0 em todos os componentes curriculares e frequência mínima de 75% das horas letivas.

In [66]:
for chunk in rag_chain.stream("Quantas são as chances de recuperar a progressão parcial?"):
    print(chunk, end="", flush=True)

As chances de recuperar a progressão parcial ocorrem quando o estudante não obtiver aprovação em até dois componentes curriculares e atender as condições estabelecidas, como a frequência mínima de 75%. O estudante terá direito a, no mínimo, três oportunidades de reensino e avaliação no ano letivo subsequente.